## Execute the steps manually in the notebook ##

### Fill table below to submit batch job

In [1]:
user_data = {
    "username": "test_user",
    "password": "test_password",
    "Client_ID": "test",
    "User_Project_ID": "sampleproject-440900",
    "User_Dataset_ID": "user_dataset",
    "Input_Table_ID": "input_table",
    "Output_Table_ID": "output_2",
    "Model": "gpt-3.5-turbo",
    "API_key": ""  
}

## Script that will submit job

In [2]:
import logging
import json
from jobPrep import retrieve_api_key, get_job_data, submit_job
from jobStatus import wait_for_completion, check_job_status

# Step 1: Retrieve API Key
logging.info("🔑 Retrieving API key...")
api_key = retrieve_api_key(user_data)

# Step 2: Prepare and Submit Job
if api_key:
    logging.info("📋 Preparing job data...")
    job_data = get_job_data(user_data)
    logging.info("📤 Submitting job...")
    job_id = submit_job(api_key, job_data)

    # Step 3: Monitor Job Status
    if job_id:
        logging.info("⏳ Monitoring job status...")
        final_status = wait_for_completion(job_id, user_data["Client_ID"], api_key,3)
        if final_status:
            logging.info("📊 Final Job Status Retrieved:")
            logging.info(json.dumps(final_status, indent=4))

2024-11-28 18:23:19,158 - INFO - 🔑 Retrieving API key...
2024-11-28 18:23:19,270 - INFO - API key retrieved successfully: 5d1f0dff-508d-46fb-9645-319df610ff65
2024-11-28 18:23:19,271 - INFO - 📋 Preparing job data...
2024-11-28 18:23:19,272 - INFO - 📤 Submitting job...
2024-11-28 18:23:19,436 - INFO - Job submitted successfully.
2024-11-28 18:23:19,437 - INFO - ⏳ Monitoring job status...
2024-11-28 18:23:19,437 - INFO - 🔎 Checking for Job Status...
2024-11-28 18:23:19,584 - ERROR - Failed to retrieve job status: 500, {"details":"'NoneType' object has no attribute 'get'","error":"Failed to retrieve job status"}

2024-11-28 18:23:19,585 - ERROR - ❌ Could not retrieve job status. Retrying...
2024-11-28 18:23:22,587 - INFO - 🔎 Checking for Job Status...
2024-11-28 18:23:22,719 - ERROR - Failed to retrieve job status: 500, {"details":"'NoneType' object has no attribute 'get'","error":"Failed to retrieve job status"}

2024-11-28 18:23:22,719 - ERROR - ❌ Could not retrieve job status. Retrying

### Manual Job Status Check

In [3]:
check_job_status(job_id=job_id, client_id=user_data["Client_ID"], api_key=api_key)

2024-11-28 18:23:35,307 - INFO - 🔎 Checking for Job Status...
2024-11-28 18:23:35,445 - INFO - Job 94163ce7-b891-4fd8-a759-da3ef437ad05 is complete. Processed 13 rows.


{'status': 'complete',
 'details': {'Client_ID': 'test',
  'Job_ID': '94163ce7-b891-4fd8-a759-da3ef437ad05',
  'current_row': 13,
  'total_rows': 13}}